<img src = 'fotos/logo_dani.jpeg'>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX 
import itertools
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

## Ficheros y rutas de entrada/salida 

In [2]:
dir_in = '../../datos/datos_originales'
file1_in = 'Dataset1.- DatosConsumoAlimentarioMAPAporCCAA.txt'
dir_out = '../../datos/datos_desarrollo'

## Carga de datos 

In [3]:
df_consumo = pd.read_csv(os.path.join(dir_in, file1_in), sep = '|').drop(['Unnamed: 10', 'Unnamed: 11'], axis = 1)
df_consumo.shape

(26634, 10)

In [4]:
df_consumo.columns = df_consumo.columns.str.lower().str.replace('[\()]', '', regex = True).str.replace(' ', '_')
df_consumo.iloc[:, 4:] = df_consumo.iloc[:, 4:].apply(lambda x: x.str.replace(',','.'))
df_consumo.iloc[:, 4:] = df_consumo.iloc[:, 4:].astype(float)

df_consumo[['producto', 'ccaa']] = df_consumo[['producto', 'ccaa']].apply(lambda x: x\
                                                                          .str.lower().str.replace('+', '', regex = False)\
                                                                          .str.replace('total', '', regex = False)\
                                                                          .str.replace('.', ' ', regex = False)\
                                                                          .str.lstrip('t')\
                                                                          .str.lstrip(' ').str.capitalize())

### Asignación de clases

In [5]:
df_consumo['month'] = np.where((df_consumo['mes']=='Enero'),1,df_consumo['mes'])
n = 1
for k in list(df_consumo.mes.unique()):
    df_consumo['month'] = np.where((df_consumo['month']==k),n,df_consumo['month'])
    n += 1
    
df_consumo['month'] = df_consumo['month'].astype('int')
df_consumo['date'] = pd.to_datetime(df_consumo.año.astype(str) + '/' + df_consumo.month.astype(str) + '/01')
df_consumo = df_consumo.drop(['month','año','mes'], axis = 1)

### Creación de columnas 

In [6]:
conditions = [
    df_consumo['date'].between("2020-03-01", "2020-06-01"),
    df_consumo['date'].between("2020-06-02", "2020-09-01"),
    df_consumo['date'].between("2020-09-02", "2020-12-10"),
    (df_consumo['date'] > "2020-12-10")
    ]

values = ['Primera ola', 'Periodo de recuperación', 'Segunda ola', 'Tercera ola']
values2 = ['Confinamiento', 'Nueva normalidad', 'Vuelta al trabajo', 'Navidad']
df_consumo['Olas'] = np.select(conditions, values, default = 'Pre-covid')
df_consumo['Motivo_ola'] = np.select(conditions, values2, default = 'Pre-covid')

In [7]:
df_consumo.to_csv(os.path.join(dir_out, 'consumo_preproc.csv'), sep = ';', index = False)